# Explicação:
Nesse arquivo há:
 - Junção e análise dos dados em catálogo usados para a tarefa de pré-treino.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
total = pd.concat([pd.read_csv(f"raw_tables/result{i}.csv") for i in range(1,9)])

In [4]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(total,test_size=0.25,random_state=2)
test = test.assign(test=1, train=0, validation=0, split="test")
train = train.assign(test=0)
train,val = train_test_split(train,test_size=0.2,random_state=2)
train = train.assign(train=1, validation=0, split="train")
val = val.assign(train=0, validation=1, split="val")

,ID,RA,DEC,FWHM_n,g_iso,i_iso,J0378_iso,J0395_iso,J0410_iso,J0430_iso,J0515_iso,J0660_iso,J0861_iso,r_iso,e_r_iso,u_iso,z_iso,SEX_FLAGS_r
0,iDR4_3_HYDRA-0028_0000625,161.413191,-26.001033,1.133159,15.249006,14.584551,16.271933,16.327374,15.678482,15.605902,15.090069,14.712759,14.562696,14.736923,0.002572,16.593010,14.510158,0
1,iDR4_3_HYDRA-0028_0000702,161.323341,-25.999224,1.134685,14.890349,14.325912,15.734061,15.690323,15.217375,15.144177,14.749876,14.447191,14.327459,14.448761,0.002187,16.205784,14.301947,0
2,iDR4_3_HYDRA-0028_0001188,162.833355,-25.988828,1.015786,14.962815,13.628279,16.594580,16.673409,15.850739,15.568675,14.659804,13.897200,13.476031,14.022722,0.001696,17.451378,13.426817,0
3,iDR4_3_HYDRA-0028_0001329,161.495867,-25.984765,1.102671,15.481448,14.534292,17.217575,16.974740,16.210407,15.969923,15.301211,14.701006,14.485750,14.764878,0.002534,17.560326,14.459332,0
4,iDR4_3_HYDRA-0028_0001342,161.543129,-25.984990,1.120254,14.344370,13.884644,15.083540,15.008025,14.593048,14.532671,14.192825,13.993132,13.889978,13.984149,0.001627,15.555052,13.884461,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21950,iDR4_3_HYDRA-0013_0015896,153.995462,-24.248249,3.363942,22.138960,21.094270,22.293330,24.255766,24.112665,22.322607,21.711924,21.769360,20.502026,21.814379,0.191051,22.848967,20.611940,0
21951,iDR4_3_HYDRA-0013_0015905,153.306442,-24.246709,0.842883,22.611246,20.178750,22.859161,23.542953,23.255526,99.000000,99.000000,20.800852,20.218887,21.268316,0.110630,99.000000,19.943070,0
21952,iDR4_3_HYDRA-0013_0015916,153.629766,-24.247760,1.409130,22.561424,19.735844,23.362900,24.486935,99.000000,99.000000,22.252182,20.951553,19.306187,21.184383,0.116990,24.120058,19.216242,0
21953,iDR4_3_HYDRA-0013_0015917,154.517728,-24.246751,2.156729,23.006046,20.825443,23.203684,99.000000,22.604918,99.000000,22.754190,21.832003,20.723244,21.813972,0.194322,23.687935,20.183940,0
